<a href="https://colab.research.google.com/github/renyuanL/_JosephLin_2023/blob/main/cnn_mlp_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#%%
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)
x=   torch.ones( 2,10)
w1=  torch.randn(10,4)

y= x@w1
y
# %%
# torch Linear
torch.manual_seed(0)
x=  torch.ones(2,10)
w2= nn.Linear(10,4, bias=False)
y= w2(x)
y

# %%
# https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#torch.nn.Conv1d

torch.manual_seed(0)
x=  torch.ones(1, 2, 10)
w3= nn.Conv1d( 2, 4,  3, padding='same')
y= w3(x)
y.shape, y
# %%
torch.manual_seed(0)
x=  torch.ones(1, 2, 10)
w4= torch.ones(4, 2, 3)
y= F.conv1d(x, w4, padding='same')
y.shape, y

# %%
# Conv2d
torch.manual_seed(0)
x=  torch.ones(5, 2, 10, 10)
w5= nn.Conv2d( 2, 4, (3,3), padding='same')
y= w5(x)
y.shape, y

# %%
# MNist
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

torch.manual_seed(0)
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

#%%
# MNist
batch_size= 64
train_loader= torch.utils.data.DataLoader(
    datasets.MNIST('data/', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
                    batch_size=batch_size, shuffle=True)

test_loader= torch.utils.data.DataLoader(
    datasets.MNIST('data/', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))  ])),
                        batch_size=batch_size, shuffle=False)

# %%
# build a CNN model to classify MNist
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1= nn.Conv2d(1, 10, kernel_size=5)
        self.conv2= nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop= nn.Dropout2d()
        self.fc1= nn.Linear(320, 50)
        self.fc2= nn.Linear(50, 10)

    def forward(self, x):
        x= F.relu(F.max_pool2d(self.conv1(x), 2))
        x= F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x= x.view(-1, 320)
        x= F.relu(self.fc1(x))
        x= F.dropout(x, training=self.training)
        x= self.fc2(x)
        return F.log_softmax(x, dim=1)
    
model= Net().to(device)
optimizer= optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

# %%
# train
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target= data.to(device), target.to(device)
        optimizer.zero_grad()
        output= model(data)
        loss= F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100.*batch_idx/len(train_loader), loss.item()
            ))

# %%
# test
def test():
    model.eval()
    test_loss= 0
    correct= 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target= data.to(device), target.to(device)
            output= model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred= output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy:{}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100.*correct/len(test_loader.dataset)
    ))

# %%
# run
for epoch in range(1, 10+1):
    train(epoch)
    test()

# %%
# build a simple mlp model to classify MNist
class MLP_Net(nn.Module):
    def __init__(self):
        super(MLP_Net, self).__init__()
        self.fc1= nn.Linear(784, 512)
        self.fc2= nn.Linear(512, 256)
        self.fc3= nn.Linear(256, 10)

    def forward(self, x):
        x= x.view(-1, 784)
        x= F.relu(self.fc1(x))
        x= F.relu(self.fc2(x))
        x= self.fc3(x)
        return F.log_softmax(x, dim=1)
    
mlp= MLP_Net().to(device)
optimizer= optim.SGD(mlp.parameters(), lr=0.01, momentum=0.5)

# %%
# train
def train(epoch):
    mlp.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target= data.to(device), target.to(device)
        optimizer.zero_grad()
        output= mlp(data)
        loss= F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
                epoch, batch_idx*len(data), len(train_loader.dataset),
                100.*batch_idx/len(train_loader), loss.item()
            ))

# %%
# test
def test():
    mlp.eval()
    test_loss= 0
    correct= 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target= data.to(device), target.to(device)
            output= mlp(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred= output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy:{}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100.*correct/len(test_loader.dataset)
    ))

# %%
# run
for epoch in range(1, 10+1):
    train(epoch)
    test()

# %%

